In [0]:
spark.read.option("header", True).csv("/mnt/datamoviehistory/bronze/2024-12-23/movie.csv").createOrReplaceTempView("v_movie_2")

spark.read.option("header", True).csv("/mnt/datamoviehistory/bronze/2024-12-30/movie.csv").createOrReplaceTempView("v_movie_3")

- Me costo 
1. tuve que cambiar el cluster a uno heredado
2. Tuve que darle permisos de Storage Blob Data Contributor en los contenedores de bronze, gold y silver

### Que Databricks asigne su propio schema
1. no es recomendable ya que cuando se manejan volumnes de datos grandes puede ser perjudicial
2.  basicamente esto es para pque Databricks le de los valores correspondientes a cada columa por ejemplo doctor_id : integer

movie_df = spark.read.option("header", True) \
    .option("inferSchema", True) \
    .csv("/mnt/datamoviehistory/bronze/movie.csv")

In [0]:
%fs
ls /mnt/datamoviehistory/bronze

Esto sirve para tener parametrizado el valor que se le dara en la enviroment creada

In [0]:
dbutils.widgets.text("p_enviroment", "2024-12-30")
v_enviroment = dbutils.widgets.get("p_enviroment")

In [0]:
dbutils.widgets.text("p_file_date", "")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
v_enviroment

Los siguientes dos conforman a reutilizar el codigo, esto para que cuando nos toque cambiar alguna ruta, no hacerlo desde cada uno de los documentos si no tener configurado algo para solo hacer un solo cambio

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/commom_functions"

In [0]:
movie_df = spark.read.option("header", True) \
    .csv(f"{bronze_folder_path}/{v_file_date}/movie.csv")

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType, DateType

In [0]:
movie_schema = StructType(
    fields = [
        StructField("movieId", IntegerType(), False),
        StructField("title", StringType(), True),
        StructField("budget", IntegerType(), True),
        StructField("homepage", StringType(), True),
        StructField("overview", StringType(), True),
        StructField("popularity", DoubleType(), True),
        StructField("yearReleaseDate", IntegerType(), True),
        StructField("releaseDate", DateType(), True),
        StructField("revenue", DoubleType(), True),
        StructField("durationTime", IntegerType(), True),
        StructField("movieStatus", StringType(), True),
        StructField("tagline", StringType(), True),
        StructField("voteAverage", DoubleType(), True),
        StructField("voteCount", IntegerType(), True)
    ]
)

In [0]:
movie_df = spark.read.option("header", True) \
    .schema(movie_schema) \
    .csv(f"{bronze_folder_path}/{v_file_date}/movie.csv")

### Seleccionar solo columnas "requeridas"

In [0]:
movie_selected_df = movie_df.select("movieId", "title", "budget", "popularity", "yearReleaseDate", "releaseDate", "revenue", "durationTime", "voteAverage", "voteCount")



### Cambiar el nombre de las columnas según lo "requerido"

In [0]:
movies_rename_df = movie_selected_df.withColumnRenamed("movieId", "movie_id") \
  .withColumnRenamed("durationTime", "duration_time") \
  .withColumnRenamed("voteAverage", "vote_average") \
  .withColumnRenamed("voteCount", "vote_count") \
  .withColumnRenamed("yearReleaseDate", "year_release_date") \
  .withColumnRenamed("releaseDate", "release_date") 

In [0]:
movies_rename_df = movie_selected_df.withColumnsRenamed({
    "movieId": "movie_id",
    "durationTime": "duration_time",
    "voteAverage": "vote_average",
    "voteCount": "vote_count",
    "yearReleaseDate": "year_release_date",
    "releaseDate": "release_date"
})

### Agregar la columna "ingrestion_date" al DataFrame

In [0]:
from pyspark.sql.functions import current_timestamp, lit

In [0]:
movies_final_df = movies_rename_df.withColumns({
  "ingestion_date": current_timestamp(),
  "enviroment": lit("production"),
  "file_date": lit(v_file_date)
})



### Escribir datos en el datalake en formato "Parquet"

In [0]:
# for item_list in movies_final_df.select("file_date").distinct().collect():
#     if (spark._jsparkSession.catalog().tableExists("movie_silver.movies")):
#         spark.sql(f"ALTER TABLE movie_silver.movies DROP IF EXISTS PARTITION (file_date='{item_list.file_date}')")


In [0]:
overwrite_partition(movies_final_df, "movie_silver", "movies", "file_date")

In [0]:
#movies_final_df.write.mode("overwrite").partitionBy("year_release_date").parquet("/mnt/datamoviehistory/silver/movies")
#movies_final_df.write.mode("overwrite").format("parquet").saveAsTable("movie_silver.movies")

# Eliminar la tabla existente
#spark.sql("DROP TABLE IF EXISTS movie_silver.movies")

# Crear nuevamente con particiones
movies_final_df.write.mode("append").partitionBy("file_date").format("parquet").saveAsTable("movie_silver.movies")

In [0]:
%fs
ls /mnt/datamoviehistory/silver/movies

In [0]:
%sql
select file_date, count(1)
from movie_silver.movies
group by file_date;